# Control allocation example for Cybership Jonny 

For the sake of testing, set `PYTHONPATH` so notebook can be run without installing the package.

In [1]:
import sys
sys.path.insert(0, "../src")

First, import the control allocation packge

In [2]:
import skadipy
import numpy as np

Now, we describe the thrusters. There are different options for thrusters. These are `Fixed`, `Aximuth`, `Vectored`.
It is defined by the position of the thruster and the direction that its directed at.
Orientation of the thruster is defined using quaternions.

Depending on the allocator choice, `extra_attributes` dictionary changes.
For Cybership Jonny, we use MinimumMagnitude and Azimuth Rate allocator. It takes `saturation_limit` and `rate_limit` values to set thrust limits for the thrusters.

In [3]:
tunnel = skadipy.actuator.Fixed(
    position=skadipy.toolbox.Point([0.3875, 0.0, 0.0]),
    orientation=skadipy.toolbox.Quaternion(
        axis=(0.0, 0.0, 1.0), radians=np.pi / 2.0),
    extra_attributes={
        "rate_limit": 0.1,
        "saturation_limit": 1.0,
        "name": "tunnel",
    }
)
port_azimuth = skadipy.actuator.Azimuth(
    position=skadipy.toolbox.Point([-0.4574, -0.055, 0.0]),
    extra_attributes={
        "rate_limit": 0.1,
        "saturation_limit": 1.0,
        "reference_angle":  np.pi / 4.0,
        "name": "port_azimuth",
    }
)
starboard_azimuth = skadipy.actuator.Azimuth(
    position=skadipy.toolbox.Point([-0.4547, 0.055, 0.0]),
    extra_attributes={
        "rate_limit": 0.1,
        "saturation_limit": 1.0,
        "reference_angle": - np.pi / 4.0,
        "name": "starboard_azimuth",
    }
)

# Put all actuators in a list and create the allocator object
actuators = [
    tunnel,
    port_azimuth,
    starboard_azimuth,
]

Now we need to describe what degrees of freedom we want to control.
To specify that, we use `ForceTorqueComponent` class.
Possible options are

- `ForceTorqueCompontent.X` for surge
- `ForceTorqueCompontent.Y` for sway
- `ForceTorqueCompontent.Z` for heave
- `ForceTorqueCompontent.K` for roll
- `ForceTorqueCompontent.M` for pitch
- `ForceTorqueCompontent.N` for yaw


In [4]:
# List all the degrees of freedom that we want to control
dofs = [
    skadipy.allocator._base.ForceTorqueComponent.X,
    skadipy.allocator._base.ForceTorqueComponent.Y,
    skadipy.allocator._base.ForceTorqueComponent.N
]

Next, we create the allocator object using the thrusters and the degrees of freedom we want to control.

In [5]:
import skadipy.allocator.reference_filters

# Create the allocator object
allocator = skadipy.allocator.reference_filters.MinimumMagnitudeAndAzimuth(
    actuators=actuators,
    force_torque_components=dofs,
    gamma=0.001,
    mu=0.01,
    rho=10,
    time_step=0.01,
    control_barrier_function=skadipy.safety.ControlBarrierFunctionType.ABSOLUTE
)

To allow change in vehicle configuration, allocation matrix can be updated using `update_allocation_matrix` method.

Finally, we can use the allocator to allocate the forces and torques to the thrusters.

In [6]:
# Compute or update the configuration matrix
allocator.compute_configuration_matrix()

Regardless of the DOF we want to control, the allocator will always return forces and torques in the body frame.
If we want to control the vehicle in the NED frame, we need to rotate the forces and torques to the NED frame.

The $\tau_{\text{cmd}}$ **must** be in the form of

$$
\tau_{\text{cmd}} = \begin{bmatrix} F_x & F_y & F_z & M_x & M_y & M_z \end{bmatrix}
$$

where $F_x, F_y, F_z$ are the forces in the body frame and $M_x, M_y, M_z$ are the torques in the body frame.

In [7]:
tau_cmd = np.array(
    [[1.0],
     [0],
     [0],
     [0],
     [0],
     [0.0]], dtype=np.float32)

# Allocate a control signal
allocator.allocate(tau=tau_cmd)

# Get the allocated force
print(f"Allocated {allocator.allocated}")

Allocated [[1.00166667e+00]
 [2.11758237e-20]
 [0.00000000e+00]
 [0.00000000e+00]
 [0.00000000e+00]
 [7.43993201e-18]]


To get commanded forces to the thrusters, we can use the actuator objects.

In [8]:
for actuator in actuators:
    print(f"{actuator.extra_attributes['name']}: {actuator.force}")

tunnel: [[-6.38889204e-08]]
port_azimuth: [[5.00833979e-01]
 [6.38005511e-06]]
starboard_azimuth: [[ 5.00832688e-01]
 [-6.31616619e-06]]
